# Composability

## Basic Composability

In [1]:
from langchain.globals import set_verbose, set_debug
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

In [3]:
chain = prompt | model | output_parser

In [4]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

In [5]:
set_verbose(False)
set_debug(True)

In [6]:
chain.invoke({"topic": "bears"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "topic": "bears"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "topic": "bears"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "Tell me a joke about bears",
          "additional_kwargs": {}
        }
      }
    ]
  }
}
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Tell me a joke about bears"
  ]
}
[llm/end] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] [1.04s] Ex

"Sure, here's a bear-themed joke for you:\n\nWhy did the bear bring a ladder to the party?\n\nBecause it heard the drinks were on the house!"

## Batching

In [7]:
set_debug(False)
chain.batch([{"topic": "bear"}, {"topic": "clowns"}])

["Why don't bears wear shoes? \n\nBecause they already have bear feet!",
 "Why don't skeletons ever become clowns?\n\nBecause they don't have the guts for it!"]

## Streaming

In [8]:
for s in chain.stream({"topic": "bears"}):
    print(s)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!


## RunnablePassthrough

In [9]:
from langchain_community.retrievers.tavily_search_api import TavilySearchAPIRetriever

retriever= TavilySearchAPIRetriever()

prompt = ChatPromptTemplate.from_template("""Answer the question based only on the context provided:

Context: {context}

Question: {question}""")

In [10]:
chain = prompt | model | output_parser

In [11]:
question = "what is langsmith"
context = "langsmith is a testing and observability platform built by the langchain team"
chain.invoke({"question": question, "context": context})

'Langsmith is a testing and observability platform developed by the Langchain team.'

In [12]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = RunnablePassthrough.assign(
    context=(lambda x: x["question"]) | retriever
) | chain

In [13]:
retrieval_chain.invoke({"question": "what is langsmith"})

'LangSmith is a platform that helps trace and evaluate language model applications and intelligent agents. It allows users to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs.'

## RunnableParallel

In [14]:
from langchain_core.runnables import RunnableParallel

In [15]:
prompt = ChatPromptTemplate.from_template("""{question}""")
simple_chain = prompt | model | output_parser

In [16]:
parallel_chain = RunnableParallel({
    "retrieved_answer": retrieval_chain,
    "simple_answer": simple_chain
})

In [17]:
parallel_chain.invoke({"question": "what is langsmith"})

{'retrieved_answer': 'LangSmith is a platform that helps trace and evaluate language model applications and intelligent agents, allowing users to move from prototype to production. It provides features such as assessing subjective qualities that automatic evaluators struggle with, sampling and validating runs, and filtering and analyzing annotations. LangSmith also simplifies the setup process and allows users to edit examples, add them to datasets, and fine-tune models.',
 'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "langsmith." It is possible that you may be referring to a specific term or concept that is not widely known or recognized. Could you please provide more context or clarify your question?'}

In [18]:
for s in parallel_chain.stream({"question": "what is langsmith"}):
    print(s)

{'simple_answer': ''}
{'simple_answer': 'Lang'}
{'simple_answer': 'smith'}
{'simple_answer': ' is'}
{'simple_answer': ' not'}
{'simple_answer': ' a'}
{'simple_answer': ' widely'}
{'simple_answer': ' known'}
{'simple_answer': ' term'}
{'simple_answer': ' or'}
{'simple_answer': ' concept'}
{'simple_answer': '.'}
{'simple_answer': ' It'}
{'simple_answer': ' does'}
{'simple_answer': ' not'}
{'simple_answer': ' have'}
{'simple_answer': ' a'}
{'simple_answer': ' specific'}
{'simple_answer': ' definition'}
{'simple_answer': ' or'}
{'simple_answer': ' meaning'}
{'simple_answer': ' in'}
{'simple_answer': ' common'}
{'simple_answer': ' usage'}
{'simple_answer': '.'}
{'simple_answer': ' It'}
{'simple_answer': ' is'}
{'simple_answer': ' possible'}
{'simple_answer': ' that'}
{'simple_answer': ' "'}
{'simple_answer': 'Lang'}
{'simple_answer': 'smith'}
{'simple_answer': '"'}
{'simple_answer': ' could'}
{'simple_answer': ' be'}
{'simple_answer': ' a'}
{'simple_answer': ' surname'}
{'simple_answer': ' 